## To Do
 - debug to get it working!!!
 - Transform the student input/label and teacher input differently
 - concatenate the labelled data and unlabelled data for 1 forward pass
 - add dropout to the student model
 - Check what losses we should be using. I think we don't use crossentropy and we use different ones for Lu and Ls

In [1]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import numpy as np
import matplotlib.pyplot as plt
import PIL

In [2]:
# taken from https://github.com/milesial/Pytorch-UNet/blob/master/unet/unet_parts.py

# Import the model
from unet import UNet

In [3]:
def transform_above_below(y, below, above):
    y1 = torch.where(y < below, 1, y)
    y2 = torch.where(y > above, 1, y1)
    return y2

In [4]:
# create some temporary dataloader to check model works
transform_size = (128, 128)

transform = transforms.Compose([
        transforms.ToTensor()
        ,transforms.Resize(transform_size, max_size=None)
])

target_transform = transforms.Compose([
        transforms.ToTensor()
        ,transforms.Lambda(lambda y: transform_above_below(y, 0.005, 0.01))
        ,transforms.Resize(transform_size, max_size=None)
        ,transforms.Lambda(lambda y: torch.where(y < 0.5, 0, 1))
])


In [5]:
# Select params:
batch_size = 4
labelled_percent = 0.25


pet_dataset = datasets.OxfordIIITPet('', transform=transform, target_transform=target_transform, download=True, target_types='segmentation')
pet_dataset_label, pet_dataset_unlabel = torch.utils.data.random_split(pet_dataset, [labelled_percent, 1 - labelled_percent])

labelled_n = len(pet_dataset_label)
batch_size_label = int(batch_size * labelled_percent)
batch_size_unlabel = batch_size - batch_size_label

data_loader_label = torch.utils.data.DataLoader(pet_dataset_label,
                                          batch_size=batch_size_label,
                                          shuffle=True,
                                          num_workers=1)
data_loader_unlabel = torch.utils.data.DataLoader(pet_dataset_unlabel,
                                          batch_size=batch_size_unlabel,
                                          shuffle=True,
                                          num_workers=1)

print(f'dataloaders should be of the same length - check should be zero: {len(data_loader_label) - len(data_loader_unlabel)}')

dataloaders should be of the same length - check should be zero: 0


In [9]:
import numpy as np
import torch
import torch.nn as nn
from torch.optim import Adam
from unet import UNet

####Initialisation####
#create 2 network
model_student = UNet(3,2)
model_teacher = UNet(3,2)
#creat the losses
sup_crit = nn.CrossEntropyLoss()
unsup_crit = nn.CrossEntropyLoss()
#optimizer
optimizer = Adam(model_student.parameters())
#other HP
epochs = 16
ramp_up = 10
consistency = 56
alpha = 0.999
gs = 0

#Weigth coef for the Unsupervised
def wt(rampup_length, current, alpha):
    if rampup_length == 0:
                return 1.0
    else:
        current = np.clip(current, 0.0, rampup_length)
        phase = 1.0 - current / rampup_length
        return float(alpha * np.exp(-5.0 * phase * phase))


#update the model_teacher weigth
def update_ema_variables(model, ema_model, alpha, global_step): 
    # Use the true average until the exponential average is more correct
    alpha = min(1 - 1 / (global_step + 1), alpha)
    for ema_param, param in zip(ema_model.parameters(), model.parameters()):
        ema_param.data.mul_(alpha).add_(1 - alpha, param.data)


##data loader - now created in previous cells
# trainloader = Data_Loader(batch_size=batch_size, shuffle=True)
for epoch in range(epochs):
    cum_loss = 0
    # for idx, (X,y) in enumerate(trainloader):
    for label_data, unlabel_data in zip(data_loader_label, data_loader_unlabel):
        
        optimizer.zero_grad()
        
        # --------------------------------
        # label iteration
        # --------------------------------
        X, y = label_data
        y = torch.squeeze(y)
        pred_stud = model_student(X)
        pred_teach = model_teacher(X)
        print(f'y shape: {y.shape}')
        print(f'student output shape: {pred_stud.shape}')

        # Calculate supervised and unsupervised losses
        Ls = sup_crit(pred_stud[80:], y[80:])
        Lu = unsup_crit(pred_stud, pred_teach)
        loss = Ls + Lu * wt(ramp_up, epoch, consistency)
        
        
        # --------------------------------
        # unlabel iteration
        # --------------------------------
        X, y = unlabel_data
        y = torch.squeeze(y)
        pred_stud = model_student(X)
        pred_teach = model_teacher(X)

        # Calculate only unsupervised losses
        Lu = unsup_crit(pred_stud, pred_teach)
        loss += Lu * wt(ramp_up, epoch, consistency)
        
        
        # --------------------------------
        # update weights
        # --------------------------------
        loss.backward()
        optimizer.step()
        cum_loss += loss
        gs += 1
        update_ema_variables(model_student, model_teacher, alpha, gs)
        print(f'global step: {gs:10d}')
    print(f'Epoch {epoch:2d} loss {(loss.item() / y.shape[0]):7.3f}')
                



KeyboardInterrupt: 